![Header](img/header_2.jpg)

# Part B: Creating a Binaural Synthesis of a Moving Sound Source
---
In this part, the topic of binaural synthesis is investigated further by simulating a moving sound source. Therefore, a trajectory of a starting airplane is created in cartesian coordinates.

The corresponding HRIRs are selected and the cross-fading between the HRIRs are implemented. Finally, the plausibility of the generated signals are investigated and evaluated.

To get an impression of the resulting auralization, listen to the final auralization result using headphones:

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)

from IPython.display import Audio
Audio("audio/auralization_with_doppler_shift.wav")

## Task 1: Trajectory of a starting airplane
---
In this task, a trajectory of a starting airplane is created in cartesian coordinates.
### Task 1.1: Creating the trajectory
---
Facing a moving sound source, the HRIRs need to be switched at defined times. Hence, it is necessary to introduce block-wise processing as well as cross-fading between those blocks. Each block with a number of samples `blocksize` corresponds to a shared position in the trajectory of the airplane.
The sampling rate `sampling_rate`, the duration of the simulated signal `duration`, the blocksize `blocksize`, the total number of blocks `n_blocks` and the neccessary positions are already defined in the cell below.

The speed of the airplane `speed`, the speed of sound `c`, the point of takeoff `start_point`, a time vector `time_vector_blocks`, containing the discrete time steps of the discrete positions, and also a time vector `time_vector` containing the discrete time steps of each sample, are already given. One meter corresponds to one unit.

Define the path of the starting airplane in cartesian coordinates by defining a vector in direction of the velocity `velocity`, and by calculating the source positions `src_pos` for each block using a linear function

$$\begin{pmatrix}p_x\\p_x\\p_z\end{pmatrix}=\begin{pmatrix}v_x\\v_y\\v_z\end{pmatrix}t+ \begin{pmatrix}b_x\\b_y\\b_z\end{pmatrix}$$

with $\vec{p}$ denoting the position for each block, $\vec{v}$ the speed of the aircraft, $t$ the time instance for the respective block and $\vec{b}$ the start position of the aircraft.

*Hint: You need to use `np.outer(...)` ([Documentation](https://numpy.org/doc/stable/reference/generated/numpy.outer.html)) to perform the outer product of two vectors.*

In [ ]:
import numpy as np
import sofa
import helper_functions as hf

# import HRIR dataset:
HRIR_path = "hrir/ITA_Artificial_Head_5x5_44100Hz.sofa"
HRIR_dataset = sofa.Database.open(HRIR_path)

# define global settings
sampling_rate = 44100
duration = 20 # seconds
total_input_samples = duration*sampling_rate

# define block size
blocksize = 1024*4
n_blocks = int(np.ceil(total_input_samples/blocksize))

# define speed of the airplane in meter per seconds
speed = 350 * 1000/3600
c = 343 # speed of sound

# define the time vector, containing the discrete time instances of each block
time_vector_blocks = np.arange(0,duration,blocksize/sampling_rate)
time_vector = np.arange(0,duration,1/sampling_rate)

# define start position of moving source
start_point = np.array((50,-1000,0))

# unit vector in direction of moving source:
direction = np.array((0,1,0.05))
direction /= np.linalg.norm(direction)

# define position, view- and up-vector of receiver
rec_pos = np.array([0,0,1.8])
rec_view_vector = np.squeeze(HRIR_dataset.Listener.View.get_values(system="cartesian"))
rec_up_vector = np.squeeze(HRIR_dataset.Listener.Up.get_values(system="cartesian"))

###### ! Solution begins here ! ######

# vector in direction of velocity:
# velocity = ...

# calculate positions via linear function (s=v*t)
# src_pos = ...

###### ! Solution ends here ! ######

### Task 1.2: Visualizing the trajectory
---
To get a visual interpretation of the situation, execute the cell below. The source positions for all time instances including the listener's view and up vector are shown in the resulting figure.

*Note: You are not supposed do do any implementation here.*

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

# create a new figure
fig = plt.figure(figsize=(10,5))
ax = plt.axes(projection='3d')

# plot the positions of the source and the receiver
ax.scatter(src_pos[:,0], src_pos[:,1],src_pos[:,2], label='Trajectory') # Students
ax.scatter(rec_pos[0],rec_pos[1],rec_pos[2], label='Receiver') # Students

# plot the view and up vector of the receiver
ax.quiver(rec_pos[0], rec_pos[1], rec_pos[2],
           rec_view_vector[0]*10, rec_view_vector[1]*10, rec_view_vector[2]*10,
           color='red', label='View vector')
ax.quiver(rec_pos[0], rec_pos[1], rec_pos[2],
           rec_up_vector[0]*20, rec_up_vector[1]*20, rec_up_vector[2]*20,
           color='green', label='Up vector')

# set the labels accordingly and activate legend
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
ax.set_zlabel('z [m]')
ax.legend();

## Task 2: Calculating the distances, azimuth and elevation angles
---
In the following, we will assume, that the airplane can be represented by a point source. The position of the source in space is defined by the trajectory above. In order to create a auralization of an airplane fly-by, we will need to include the perceived loudness change due to the distance variation and the correct HRTF for the angles corresponding to the airplane's position relative to the receiver. The preceived loudness change is due to the reduction of sound pressure according to $p(r) \sim 1/r$, that is the inverse distance law.
In order to apply the inverse distance law and also to select the respective HRIR for each block, the distances and azimuth/elevation angles have to be calculated.
### Task 2.1: Angle between two vectors
---
Implement the function `angle_between_vectors(v1, v2, deg=True)`.
For calculating an angle between two vectors, the following dependencies hold true:
$$ \cos(\theta)  = \frac{\vec{a} \cdot \vec{b}}{|\vec{a}| |\vec{b}|}$$

$$ \sin(\theta)  = \frac{\vec{a} \times \vec{b}}{|\vec{a}| |\vec{b}|}$$

$$ \tan(\theta)  = \frac{\sin(\theta)}{\cos(\theta)}$$

*Hint: It is not sufficient to only use the dot product here, as the sign of the angle is of interest. Use all three equations above for the calculation!*

In [ ]:
def angle_between_vectors(v1, v2, deg=True):
    """
    Returns the angle of two n-dimensional vectors.

    Parameters
    ----------
    v1 : numpy.ndarray
        First vector.
    v2 : numpy.ndarray
        Second vector.

    Returns
    -------
    angle : numpy.ndarray
        The angle between the given vectors.
    """
###### ! Solution begins here ! ######

    # ...
        
###### ! Solution ends here ! ######
    if deg:
        angle = np.degrees(angle)
    return angle

### Task 2.2: Calculating the distances, azimuth and elevation angles
---
In order to apply the inverse distance law (1/r) and also to select the respective HRIR for each block, the distances and azimuth/elevation angles have to be calculated.

1. Calculate the distances of the receiver and the positions of the aircraft using the euclidean norm, provided by `np.linalg.norm()` ([Documentation](https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html)) and store them in the array `distances`.

2. Use the previously implemented function `angle_between_vectors(v1, v2)` to calculate the azimuth and elevation angles beween the up/view-vector and the source position vectors. A simple way to do this is to project the vectors to the xy- and xz-plane respectively and use the resulting two-dimensional vectors to calculate the angle. Store the results in the arrays `azimuth` and `elevation`.

*Note: The resulting angles are only valid for the static receiver position. Due to the scope of this lab, a valid implementation of the rotation of the listener is omitted.*

In [ ]:
###### ! Solution begins here ! ######

# calculate the distances of the source and receiver for each block
# ...

# projection of vectors to the xy and yz plane
# ...

# calculate the azimuth and elevation angles using the prevously implemented function
# azimuth = ...
# elevation = ...

###### ! Solution ends here ! ######

## Task 3: Defining the excitation signal
---
In this task, the excitation signal of the source/airplane is defined. Take a look at the excitation's spectrogram and listen to it using the widget below. Describe the components of the excitiation signal.

*Note: You are not supposed do do any implementation here.*

In [ ]:
from scipy.io import wavfile

# define the excitation signal as a mixture of white Gaussian noise
# and a sine mixture with a fundamental frequency of 880 Hz and some harmonics
f0 = 880
alpha = 0.2
excitation = (np.random.standard_normal(total_input_samples)
              + alpha * np.sin(2*np.pi*f0*time_vector)
              + alpha * np.sin(2*np.pi*2*f0*time_vector)
              + alpha * np.sin(2*np.pi*3*f0*time_vector)
              + alpha * np.sin(2*np.pi*4*f0*time_vector)
              + alpha * np.sin(2*np.pi*5*f0*time_vector)
              + alpha * np.sin(2*np.pi*6*f0*time_vector))

# filter and normalize the excitation signal
excitation = hf.butter_lowpass_filter(excitation, sampling_rate, cutoff=2000, order=3)
excitation = hf.normalize(excitation)

In [ ]:
# plot a spectrogram of the excitation:
plt.figure(figsize=(16, 6))
plt.specgram(excitation/excitation.max(), NFFT=1024, Fs=44100, mode='magnitude', scale='dB', vmax=-20, vmin=-150)
plt.xlabel('Time [s]')
plt.ylabel('Frequency [Hz]')
plt.ylim(20,15000)
plt.colorbar()
# export the excitation signal as a wave file and play it back via IPython
hf.write_wav(excitation, 'output/excitation.wav', 44100)
Audio("output/excitation.wav")

In [ ]:
# Write down your answer here:

# ...

## Task 4: Cross-fading and evaluation
---
For a continuous auralization it is required to implement cross-fading between each block to avoid artifacts. Otherwise the discontinuities in the discretely sampled HRTF set will be clearly audible as jumps in the perceived position of the airplane.
### Task 4.1: Dividing the signal into blocks
The function `create_frames(input_sequence, hop, window_size)` in the cell below enables to split the `input_sequence` into frames of a given `window_size`. The function description contains a visual explanation of the process, which might be helpful for understanding the procedure.

*Note: You are not supposed do do any implementation here.*

In [ ]:
def create_frames(input_sequence, hop, window_size):
    """
    Splits the input sequence in overlapping frames and
    stores these frames into a matrix.

    |------------------Input vector-----------------------|

    |------1------|
    |-hop-|------2------|
                |------3------|
                      |------4------|
                            ...

    Index            Frame
      1         |------1------|
      2         |------2------|
      3         |------3------|
      4         |------4------|
     ...              ...

    Parameters
    ----------
    input_sequence : numpy.ndarray
        Input sequence array.
    hop : integer
        Hopsize between consecutive frames.
    window_size : integer
        Size of each frame.

    Returns
    -------
    frames : numpy.ndarray
        Output array containing the frames.
    """

    # find the maximum number of slices that can be obtained
    number_slices = int(np.floor((input_sequence.size-window_size)/hop))

    # truncate if needed to get only an integer number of hop
    input_sequence = input_sequence[0:int(number_slices*hop+window_size)];

    # create a matrix with time slices
    frames = np.zeros((int(np.floor(input_sequence.size/hop)), window_size))
    
    # fill the matrix
    for idx in range (0,number_slices):
        index_time_start = int(idx*hop)
        index_time_end = int(idx*hop + window_size)
        frames[idx,:] = input_sequence[index_time_start:index_time_end]
        
    return frames

### Task 4.2: Selecting the HRIRs and implementing the cross-fading between the processed blocks
---
Implement the convolution with the HRIRs and the cross-fading between the convolved blocks.

1. Use the function `create_frames` to create the frames and store them in the array `blocks`. The window size corresponds to the block size multiplied with the overlap factor, which is already defined in the cell below. 

2. Complete the main processing loop. Select the respective HRIR and apply the inverse distance law to the current frame.

3. Apply a Hanning window to the frame (cross-fade) using the predefined array `hanning`.

4. Convolve the frame with the respective HRIR. For this use the function `signal.oaconvolve(...)` ([Documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.oaconvolve.html)) and store the results in two arrays. *Hint: Use 'same' as mode and specify the correct axis.* In advance, you need to stack the monaural input to a "double-mono" array using `np.vstack(...)` ([Documentation](https://numpy.org/doc/stable/reference/generated/numpy.vstack.html)). You may copy your code from part A of the lab to do this.

5. Add the result to the output array using an overlap-add principle. Pay attention to the correct indexing.

In [ ]:
from scipy import signal
from ipywidgets import IntProgress
from IPython.display import display

# initialize progress bar:
progress_bar = IntProgress(min=0, max=int(n_blocks)-1, description='Processing:')
display(progress_bar)

# define overlap factor for windowing:
overlap_factor = 4

# pre-define hanning window:
hanning = np.hanning(overlap_factor * blocksize)

###### ! Solution begins here ! ######

# create the overlapping blocks:
# blocks = ...

###### ! Solution ends here ! ######


# initialize the output matrix to store the output blocks in
out = np.zeros([2, sampling_rate*(duration+1)])

# main block processing loop
for idx in range(0, int(n_blocks)-1):
    
###### ! Solution begins here ! ######

    # select the respective HRIRs:    
    # HRIR = ...
    
    # select the start index of the current block:       
    start = int((idx)*blocksize)
    
    # select the current input frames and apply the hanning window and also the 1/r law:           
    
    # ...
    # input_frame_windowed = ...
    
    # convolve the input frames with the respective HRIR    
    # direct = ...

    # apply the overlay add operation
    # out[:,start:start+direct.shape[-1]] = ...
    
###### ! Solution ends here ! ######

    progress_bar.value += 1

#normalize the output matrix
auralization_without_reflection = hf.normalize(out)


### Task 4.3: Playback and evaluation of audiblity
---
Evaluate the plausibility of the auralization and write down your observations. Name possibilities to improve the result. Which acoustical phenomena are not considered yet?

Try to find an explanation for the steplike artifacts that can be obtained from the spectrogram. 

In [ ]:
plt.figure(figsize=(16, 6))
plt.specgram(auralization_without_reflection[0], NFFT=1024,Fs=44100, mode='magnitude', scale='dB', vmax=-20, vmin=-180);
plt.colorbar()
plt.xlabel('Time [s]')
plt.ylabel('Frequency [Hz]')
plt.ylim(20,15000)
hf.write_wav(auralization_without_reflection,'output/auralization_without_reflection.wav', 44100)
Audio("output/auralization_without_reflection.wav")

In [ ]:
# Write down your answer here:

# ...

# Part C: Floor-Reflection and the Doppler Shift
---
In this part, a floor reflection is considered by adding a source mirrored at the xy-plane. This method is commonly known as the image source  or mirror source algorithm. Later, a simplified auralization of the Doppler shift is given and evaluated.

## Task 1: Calculateing the positions, distances, azimuth and elevation angles of the mirrored source
---
1. Calculate the posisitons of the image source by mirroring it at the xy-plane and store them in the array `mirror_src_pos`. The image source representing the reflection. The position of the image source is calculated using a transformation vector and performed as an elementwise multiplication with `src_pos`. See code below.

2. Calculate the distances of the receiver and the positions of the mirror source using the euclidean norm, provided by `np.linalg.norm()` ([Documentation](https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html)) and store them in the array `mirror_distances`.

3. Use the previously implemented fuction `angle_between_vectors(v1, v2)` to calculate the azimuth and elevation angles beween the up/view-vector and the source position vectors. A simple way to do this is to project the vectors to the xy- and xz-plane respectively and use the resulting two-dimensional vectors to calculate the angle. Store the results in the arrays `mirror_azimuth` and `mirror_elevation`.

4. Calculate the time difference by subtracting the two distance vectors and applying the speed of sound. Store the absolute value of the result in the array `time_difference`.

*Note: The resulting angles are only valid for the static receiver position. Due to the scope of this lab, a valid implementation of the rotation of the listener is omitted.*

In [ ]:
###### ! Solution begins here ! ######

# 1. calculate the positions of the source, mirrored at the floor (xy-plane):
transformation_vector = np.array((1,1,-1))
mirror_src_pos = src_pos * transformation_vector

# 2. calculate the distances of the mirror source to the receiver
# mirror_distances = ...

# projection of vectors to the xy and yz plane
# ...

# 3. calculate the azimuth and elevation angles 
# ...

# 4. calculate the delay beween direct and reflected sound wave (t=s/v)
# ...

###### ! Solution ends here ! ######

## Task 2: Cross-fading and evaluation
---
Extend the cross-fading loop and consinder the reflection.

### Task 2.1: Selecting the HRIRs and implementing the cross-fading between the processed blocks
---
Implement the convolution with the HRIRs and the cross-fading between the convolved blocks.

1. Use the implemented code from task 4.2 of the previous part and extend it in order to process the floor reflection.

2. The floor reflection arrives later at the receiver than the direct sound. Thus, a delay is neccessary. Use the vector `time_difference` to calculate the number of samples for the delay. You are not expected to do an interpolation between the samples. For this task it is sufficient to floor the delay to integers.

*Hint: Use the function np.pad(...) ([Documentation](https://numpy.org/doc/stable/reference/generated/numpy.pad.html)) to implement the delay. Also, the axes of the resulting arrays need to be aligned in order to perform elementswise calculations.*

In [ ]:
# initialize progress bar:
progress_bar = IntProgress(min=0, max=int(n_blocks)-1, description='Processing:')
display(progress_bar)

# define overlap factor for windowing:
overlap_factor = 4

# pre-define hanning window:
hanning = np.hanning(overlap_factor * blocksize)

# create the overlapping blocks:
blocks = create_frames(excitation, blocksize, blocksize*overlap_factor)

# initialize the output matrix to store the output blocks in
out = np.zeros([2, sampling_rate*(duration+1)])

# main block processing loop
for idx in range(0, int(n_blocks)-1):  
    
###### ! Solution begins here ! ######

    # select the respective HRIRs:    
    HRIR = hf.get_HRIR_at_direction(HRIR_dataset, azimuth[int(idx)], elevation[int(idx)])
    # mirror_HRIR = ...
    
    # select the start index of the current block:       
    start = int((idx)*blocksize)

    # select the current input frame 
    # ...
    
    # apply the hanning window and at the same time the 1/r law:               
    # input_frame_windowed = ...
    # mirror_input_frame_windowed = ...
    
    # convolve the mirror input frames with the respective HRIR    
    # ...
        
    # convolve the input frames with the respective HRIR    
    # ...
    
    # calculate the delay of the reflection in samples:
    shift = int(np.floor(time_difference[idx]*sampling_rate))

    # apply the overlap add operation separately for each the direct sound and the reflection
    # out[:,start:start+direct.shape[-1]] = ...
    # out[:,start+shift:start+shift+reflection.shape[-1]] = ...

###### ! Solution ends here ! ######

    progress_bar.value += 1

#normalize the output matrix
auralization_with_reflection = hf.normalize(out)

### Task 2.2 Playback and evaluation of audiblity
---
Compare the resulting audio with the previous implementation and write down your observations. Is there a change in  perceived distance when adding reflections? Try to find an explanation for the audible effects.

In [ ]:
plt.figure(figsize=(16, 6))
plt.specgram(auralization_with_reflection[0], NFFT=1024,Fs=44100, scale='dB', vmin=-180)
plt.colorbar()
plt.xlabel('Time [s]')
plt.ylabel('Frequency [Hz]')
plt.ylim(20,15000)
hf.write_wav(auralization_with_reflection, 'output/auralization_with_reflection.wav', 44100)
Audio("output/auralization_with_reflection.wav")

In [ ]:
# Write down your answer here:

# ...

## Task 3: Doppler shift and evaluation
---
This task gives a simple auralization of the Doppler shift.
### Task 3.1: Playback and evaluation of audiblity
---
Compare the resulting audio with the previous implementation and write down your observations. Is the plausibility of the auralization enhanced?

In [ ]:
# apply the doppler effect by using linear interpolation and the delay
src_pos_cont = start_point+np.outer(time_vector, velocity)
distances_cont = np.linalg.norm(src_pos_cont - rec_pos, axis=-1)
delay_cont = distances_cont / c
auralization_with_doppler_shift = hf.linear_interpolation(
    auralization_with_reflection[:,0:delay_cont.shape[0]], delay_cont)

In [ ]:
plt.figure(figsize=(16, 6))
plt.specgram(auralization_with_doppler_shift[0], NFFT=1024,Fs=44100, scale='dB', vmin=-180)
plt.colorbar()
plt.xlabel('Time [s]')
plt.ylabel('Frequency [Hz]')
plt.ylim(20,15000)
hf.write_wav(auralization_with_doppler_shift, 'output/auralization_with_doppler_shift.wav', 44100)
Audio("output/auralization_with_doppler_shift.wav")

In [ ]:
# Write down your answer here:

# ...